# Get Vasari letters from Fondazione Memofonte (https://www.memofonte.it/ricerche/giorgio-vasari/) with Beautiful Soup

#### based on Jeri Wieringa, "Intro to Beautiful Soup," The Programming Historian 1 (2012), https://doi.org/10.46430/phen0008.

In [1]:
import urllib.request, urllib.error, urllib.parse
import csv
from bs4 import BeautifulSoup
import html2text
import os
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
data_path = 'data/letters/'

## Iterate over all URLs and save as .html

#### make list of URLs

In [7]:
# letter-id ranging from 1 to 1151

url_list = []
url_base = "https://www.memofonte.it/home/ricerca/singolo_17.php?id="
for i in range (1151):
    url_list.append(url_base+ str(i+1))
    
len(url_list)

1151

### save all as .html

In [9]:
count = 0
for i in url_list:
    count+=1
    response = urllib.request.urlopen(i)
    webContent = response.read()
    #filename = 'data/results/letters/html/letter-'+str(i)'.html'
    f = open('data/letters/letter-'+str(count)+'.html', 'wb')
    f.write(webContent)
    f.close()
    

## Open all .html, make Beautiful Soups, extract information, save all letters in a single CSV

In [14]:
list_of_htmls = os.listdir('data/letters')
len(list_of_htmls)
#list_of_htmls

1151

In [15]:
# get rid of tags without Beautiful Soup
# because there is a space " " missing after some <br/> tags

def stripTags(pageContent):
    pageContents = str(pageContent)

    inside = 0
    text = ''
    storage = " "

    for char in pageContents:
        if (char == '<' and storage == " "):
            inside = 1
        elif (char == '<' and storage != " "):
            inside = 1
            text += " "     
        elif (inside == 1 and char == '>'):
            inside = 0
            #text += " "
        elif inside == 1:
            continue
        else:
            text += char
        storage = char

    return text

### Make one CSV File for all letters

In [16]:
# make CSV File
# changed filename after first run from letters.csv to letters_x.csv
csv_file = csv.writer(open('data/all_letters.csv', "w"))

# Write column headers as the first line
csv_file.writerow(["Numero d'ordine", "Data", "Intestazione", "Segnatura", "Fonte", "Bibliografia"]) 

64

#### iterate over all .html files, save as csv

In [17]:
# open all letter.html, make soup, extract information, collect it in a list, append list as a row to csv_file

for i in list_of_htmls:
    csv_row = list() # make new row
    soup = BeautifulSoup(open('data/letters/'+i), "lxml") # make soup
    p_array = soup.find_all('p') # take the information one by one 
    csv_row.append(int(p_array[0].text[(len('Numero d\'ordine:')+1):])) # append each to the list
    csv_row.append(p_array[1].text[(len('Data:')+1):])
    csv_row.append(p_array[2].text[(len('Intestazione:')+1):])
    csv_row.append(p_array[3].text[(len('Segnatura:')+1):])
    #csv_row.append(p_array[4].text[(len('Fonte:')+1):])
    x = (p_array[4])
    formatted_text = stripTags(str(x)[12:]) # remove html tags without beautiful soup
    csv_row.append(formatted_text.replace("\xa0", ''))
    
    csv_row.append(p_array[5].text[(len('Bibliografia:')+1):])
    csv_file.writerow(csv_row)

## open or save CSV-file as dataframe

In [18]:
filename = 'data/all_letters.csv'

dataframe = pd.read_csv(filename) # open csv
#dataframe.to_csv(filename, index = False, header = True) # save as csv

dataframe["Fonte"][1146]

'Reverendissimo monsignore patron mio, io mando alla Signoria Vostra Reverendissima questa inclusa ch’è della Comunità d’Arezzo; la Signoria Vostra sia contenta, spedita ch’ella sarà, di rimandamela in mano, perché importa alla mia Comunità e ve ne arò obligo. Io vi ho scritto e Sua Eccellenzia doverrà deliberare quanto gli piacerà delle cose che bosogniano per qua per servizio suo. Ebbi un’ultima sua col sugel grosso, farò quanto m’è comesso e tuttavia si atende; così potessi voi con tutta la corte goder quel ch’è fatto senza avere a star ne’ luoghi che son disfatti, pure il faro che vi conduce lo debbe saper lui; andrò pregando Dio che vi riconduca a noi e che vi dia pazienzia che questo santo ch’è oggi nel giorno, che io scrivo, i diavoli gniene scossono coi bastoni; dove il nostro Bernia dice in questi due versi: a giorni diciassette l’Alcionio fu bastonato come santo Antonio. E con questo fo fine, restando al suo comando. Di vostra Signoria reverendissimo servitore Giorgio Vasari 

In [19]:
dataframe

,Numero d'ordine,Data,Intestazione,Segnatura,Fonte,Bibliografia
0,1051,26 12 1572,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASA, AV, 14 (XLVIII), cc. 146-148.",Molto Magnifico messer Giorgio. Ho ricevuto la...,"Frey 1930, pp. 736-737."
1,791,24 12 1569,GIORGIO VASARI IN FIRENZE A ORDINE DI SANTO ST...,"ASPI, Carte Bonaini, IX.",Illustrissimi Signori miei observandissimi. Co...,"Frey 1930, pp. 476-477."
2,284,20 04 1559,GABRIEL FIAMMA IN PADOVA A GIORGIO VASARI IN F...,"ASA, AV, 13 (XLVII), cc. 53, 59.",Magnifico Signor mio osservandissimo. Ho scrit...,"Frey 1923, pp. 513-514."
3,1114,1 08 1573,GIORGIO VASARI IN FIRENZE AI DEPUTATI ALLE FAB...,"AAA, filza CC, Registro di lettere, i, 1569, I...","Molto magnifici Signori Deputati, Signori mia ...","Frey 1930, pp. 810-811."
4,904,3 09 1571,GUGLIELMO SANGALLETTI IN ROMA A GIORGIO VASARI...,"ASA, AV, 15 (XLIX), cc. 174, 177.",Molto Magnifico Signor Cavaliere. La presente ...,"Frey 1930, pp. 599-600."
...,...,...,...,...,...,...
1146,356,1 01 1562,GIORGIO VASARI IN FIRENZE A JACOPO GUIDI,"ASF, Carte Guidi, c. 356.","Reverendissimo monsignore patron mio, io mando...","Palli D’Addario 1985, pp. 379-380 (post 2 giug..."
1147,585,18 06 1565,GIORGIO VASARI IN FIRENZE A VINCENZO BORGHINI...,"ASF, CdA, II, I, n. 30.",Reverendo Signor mio. S’è riceuto la vostra le...,"Gaye 1839-1840, III, p. 185; Milanesi 1878-18..."
1148,839,29 08 1570,VINCENZO BORGHINI IN POPPIANO A GIORGIO VASARI,"ASA, AV, 14 (XLVIII), c. 70.",Messer Giorgio mio. Io vi mando una lunga dice...,"Frey 1930, pp. 522-523."
1149,993,6 06 1572,BARTOLOMMEO BUSSOTTO IN ROMA A GIORGIO VASARI ...,"ASA, AV, 10 (XLIV), cc. 83, 86.",Molto magnifico Signor come fratello. Scrissi ...,"Frey 1930, pp. 681-682."


## get rid of nan-values

In [20]:
# function to check if 'nan'
def isnan(x):
    return x != x

In [21]:
count = 0
for i in range(len(dataframe)): # check column by column if isnan and replace with '0'
    
    if isnan(dataframe['Intestazione'][i]): 
        print('Intestazione, ', count)
        (dataframe['Intestazione'][i]) = '0'
        
    if isnan(dataframe['Segnatura'][i]):
        #print('Segnatura', count)
        (dataframe['Segnatura'][i]) = '0'
        
    if isnan(dataframe['Fonte'][i]):
        #print('Fonte', count)
        (dataframe['Fonte'][i]) = '0'
        
    if isnan(dataframe['Bibliografia'][i]):
        #print('Bibliografia', count)
        (dataframe['Bibliografia'][i]) = '0'
    
    count+=1

Intestazione,  17
Intestazione,  101
Intestazione,  268
Intestazione,  368
Intestazione,  491
Intestazione,  569
Intestazione,  617
Intestazione,  703
Intestazione,  740
Intestazione,  928
Intestazione,  985


<ipython-input-21-587cb89d412f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  (dataframe['Intestazione'][i]) = '0'
<ipython-input-21-587cb89d412f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  (dataframe['Segnatura'][i]) = '0'
<ipython-input-21-587cb89d412f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  (dataframe['Bibliografia'][i]) = '0'


'nan' count in total: 11 rows with nan values 
--> replace with '0' and collect index for documentation (row-indices found: 17, 101, 268, 368, 491, 569, 617, 703, 740, 928, 985)

# Distinguish letters by Vasari himself, copied by his nefew and authorship candidates

example for signature of letter copied by nefew (called Giorgio Vasari the younger) 
BRF, 2354, cc. 12v-14, copia di Giorgio Vasari il Giovane.

number of letters filtered at http://vasariscrittore.memofonte.it/archivio
390

In [3]:
filename = 'data/all_letters.csv'
#dataframe = pd.read_csv(filename) # open csv

In [23]:
dataframe['Intestazione'] = [i.strip(' ') for i in dataframe['Intestazione']]

In [24]:
# check who has sent how many letters

sender = dataframe['Intestazione']
store = []
for i in sender:
    store.append(i[:15])    
for i in set(store):
    print(i, store.count(i))

ALESSANDRO MUSO 2
DEPUTATI ALLE F 1
GIOVAN MARIA DE 1
FRANCESCO VINTA 2
LEONARDO MARINO 1
IPPOLITO OLIVET 19
BARTOLOMMEO BUS 7
BENEDETTO LOMEL 3
JACOPO GUIDI IN 5
MICHELE DEGLI A 1
FRANCESCO ALCIA 1
GIORGIO VASARI  404
PIER JACOPO, MA 2
PIER DONATO CES 9
FEDERICO DI MON 12
GABRIEL FIAMMA  16
MINIATO PITTI I 23
BERNARDETTO MIN 66
VINCENZO BORGHI 136
TIBERIO CALCAGN 1
GIULIO RICASOLI 1
LODOVICO DOMENI 1
DON ANTONIO IN  1
PIERGIOVANNI AL 1
PIETRO ARETINO  10
JACOPO APPIANI  1
PAOLO GIOVIO IN 18
PIETRO VASARI I 2
BERNARDINO E FE 1
COSIMO BARTOLI  73
GUGLIELMO SANGA 108
MICHELE BONELLI 3
FRANCESCO DE’ M 7
ALESSANDRO STRO 2
VESCOVO CORNARO 1
GIOVANNI BATTIS 12
PIETRO CAMAIANI 5
COSIMO DE’ MEDI 43
AVERARDO SERRIS 1
SIGNORE DI PIOM 2
BATTISTA LORENZ 1
LAMBERT LOMBARD 1
BARTOLOMEO CONC 14
0 11
GIOVANNI GORI E 1
FRA’ STEFANO IN 2
PIO V A GIORGIO 1
CHIAPPINO VITEL 1
ARCIVESCOVO DI  1
PIERFRANCESCO G 6
FILIPPO BUONCOM 14
ALESSANDRO FARN 1
GABRIELE PALEOT 1
LORENZO SABATIN 1
DON SAMUELE IN  1
DOMIN

### split and label dataframe of letters for different involved authorship candidates

Cosimo Bartoli and Pierfrancesco Giambullari involved as Co-Authors after Gerd Blum (https://archiv.ub.uni-heidelberg.de/artdok/1948/1/Blum_Provvidenza_e_progresso_2010.pdf) 

Pierfrancesco Giambullari, Cosimo Bartoli, Giovan Battista Gelli und Carlo Lenzoni involved in the preparation of the edition of 1550; massive contribution of Vincenzio Borghinis concerning the indices and growing involvement over the following decennies (Alessandro Nova: Vasari versus Vasari, S.151)

In [6]:
# get Pierfrancesco Giambullari, Cosimo Bartoli, Vincenzio Borghini, Minerbetti Bernadetto, Sangaletti Guglielmo

# examples of the signatures copied from http://vasariscrittore.memofonte.it/archivio# 
# COSIMO BARTOLI IN FIRENZE A GIORGIO VASARI IN ROMA (73 letters)
# VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI IN ROMA (131 letters)
# PIERFRANCESCO GIAMBULLARI IN FIRENZE A GIORGIO VASARI IN AREZZO (6 letters)
# MINERBETTI BERNARDETTO 108 letters
# SANGALLETTI GUGLIELMo 66 letters
# GIORGIO VASARI 363
# GIORGIO VASARI copied by his nephew 41

Borghini = list() # collect whole entries written by Vincenzo Borghini
Giambullari = list() # collect Pierfrancesco Giambullari
Bartoli = list() # collect Cosimo Bartoli
Minerbetti = list() # collect MINERBETTI BERNARDETTO
Sanga = list()  # collect SANGALLETTI GUGLIELMO
Vasari = list() # Giorgio Vasari 363
VasariC = list() # Giorgio Vasari copies 41

new_column = list()

for i in range(len(dataframe)):

    if dataframe['Intestazione'][i].startswith('VINCENZO BORGHINI'): # VINCENZO BORGHINI
        Borghini.append(list(dataframe.iloc[i]))
        new_column.append('Borghini')
    
    elif 'COSIMO BARTOLI' in dataframe['Intestazione'][i]: # COSIMO BARTOLI
        Bartoli.append(list(dataframe.iloc[i]))
        new_column.append('Bartoli')
    
    elif 'PIERFRANCESCO GIAMBULLARI' in dataframe['Intestazione'][i]: # Pierfrancesco Giambullari
        Giambullari.append(list(dataframe.iloc[i]))
        new_column.append('Giambullari')
        
    elif dataframe['Intestazione'][i].startswith('BERNARDETTO MIN'): # BERNADETTO MINERBETTI
        Minerbetti.append(list(dataframe.iloc[i]))
        new_column.append('Minerbetti')
        
    elif dataframe['Intestazione'][i].startswith('GUGLIELMO SANGA'): # VINCENZO BORGHINI
        Sanga.append(list(dataframe.iloc[i]))
        new_column.append('Sanga')
        
    elif dataframe['Intestazione'][i].startswith('GIORGIO VASARI') and 'copia' in dataframe['Segnatura'][i]: # VINCENZO BORGHINI
        VasariC.append(list(dataframe.iloc[i]))
        new_column.append('VasariC')
    
    elif dataframe['Intestazione'][i].startswith('GIORGIO VASARI') and 'copia' not in dataframe['Segnatura'][i]: # filter out copies        
        Vasari.append(list(dataframe.iloc[i]))
        new_column.append('Vasari')
        
    else:
        new_column.append('other')
        
dataframe['author'] = new_column
print('Giambullari: ', len(Giambullari), 'Borghini: ', len(Borghini), 'Bartoli: ', len(Bartoli), 'Sanga: ', len(Sanga), 'Minerbetti: ', len(Minerbetti), 'Vasari: ', len(Vasari), 'VasariC: ', len(VasariC))

Giambullari:  6 Borghini:  136 Bartoli:  73 Sanga:  108 Minerbetti:  66 Vasari:  363 VasariC:  41


#### make single dataframes

In [8]:
columns = ["Numero d'ordine", "Data", "Intestazione", "Segnatura", "Fonte", "Bibliografia", 'author']

In [9]:
df_Giambullari = pd.DataFrame(Giambullari, columns = columns)
df_Borghini = pd.DataFrame(Borghini, columns = columns)
df_Bartoli = pd.DataFrame(Bartoli, columns = columns)
df_Minerbetti = pd.DataFrame(Minerbetti, columns = columns)
df_Sanga = pd.DataFrame(Sanga, columns = columns)
df_Vasari = pd.DataFrame(Vasari, columns = columns)
df_Vasari_c = pd.DataFrame(VasariC, columns = columns)

## save dataframes as csv files

In [10]:
data_path = 'data/grouped_letters/'

In [11]:
df_Giambullari.to_csv(data_path + 'Giambullari.csv', index = False, header=True)
df_Borghini.to_csv(data_path + 'Borghini.csv', index = False, header=True)
df_Bartoli.to_csv(data_path + 'Bartoli.csv', index = False, header=True)
df_Minerbetti.to_csv(data_path + 'Minerbetti.csv', index = False, header=True)
df_Sanga.to_csv(data_path + 'Sanga.csv', index = False, header=True)
df_Vasari.to_csv(data_path + 'Vasari.csv', index = False, header=True)
df_Vasari_c.to_csv(data_path + 'VasariC.csv', index = False, header=True)

### open csv as dataframes

In [43]:
Gia = 'Giambullari.csv'
Bor = 'Borghini.csv'
Min = 'Minerbetti.csv'
Bar = 'Bartoli.csv'
San = 'Sanga.csv'

df_Giambullari = pd.read_csv(data_path+Gia)
df_Borghini = pd.read_csv(data_path+Bor)
df_Bartoli = pd.read_csv(data_path+Bar)
df_Minerbetti = pd.read_csv(data_path+Min)
df_Sanga = pd.read_csv(data_path+San)

# open or save files

In [12]:
filename = 'data/all_letters.csv'
#dataframe = pd.read_csv(filename) # open csv
dataframe.to_csv(filename, index = False, header = True) # save as csv

#dataframe

# END OF NOTEBOOK